## Assignment 1
### Stochastic Simulation

Team:
- Yuxin
- Marcel
- Koen

In [ ]:
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
from sympy import isprime

Define the function that let's us randomly sample in three different ways:
- Pure Random Sampling
- Latin Hypercube Sampling
- Orthogonal Sampling

In [ ]:
# Define the random sampling function
def random_sample(n, x_range=(0,10), y_range=(0,10), sampler='pure_random_sampling'):
    if sampler == 'pure_random_sampling':
        # Pure random sampling
        x_coords = np.random.uniform(x_range[0], x_range[1], n)
        y_coords = np.random.uniform(y_range[0], y_range[1], n)
        points = np.stack((x_coords, y_coords), axis=-1)

    elif sampler == 'latin_hypercube_sampling':
        # Latin Hypercube Sampling
        lhc = qmc.LatinHypercube(d=2)
        points = lhc.random(n)
        points[:, 0] = points[:, 0] * (x_range[1] - x_range[0]) + x_range[0]
        points[:, 1] = points[:, 1] * (y_range[1] - y_range[0]) + y_range[0]

    elif sampler == 'orthogonal_sampling':
        # Orthogonal Sampling: Check if n is the square of a prime number
        sqrt_n = int(np.sqrt(n))
        if sqrt_n**2 != n or not isprime(sqrt_n):
            raise ValueError("n must be the square of a prime number for orthogonal sampling.")
        # Use Latin Hypercube Sampling for orthogonal sampling
        lhc = qmc.LatinHypercube(d=2)
        points = lhc.random(n)
        points[:, 0] = points[:, 0] * (x_range[1] - x_range[0]) + x_range[0]
        points[:, 1] = points[:, 1] * (y_range[1] - y_range[0]) + y_range[0]


    else:
        raise ValueError("Invalid sampler specified.")
    
    return points


In [ ]:
# Define Mandelbrot set calculation function
def mandelbrot(point, max_iter=1000):
    z = 0
    for i in range(max_iter):
        z = z**2 + point
        if abs(z) > 2:
            return i
    return max_iter


In [ ]:
# Plotting function for the Mandelbrot set
def plot_mandelbrot(n=10000, x_range=(-2,1), y_range=(-1.5, 1.5), max_iter=1000, sampler='pure_random_sampling'):
    points = random_sample(n, x_range, y_range, sampler)
    complex_points = points[:, 0] + 1j * points[:, 1]
    iterations = np.array([mandelbrot(point, max_iter) for point in complex_points])

    # Separate points inside and outside the Mandelbrot set
    in_set = iterations == max_iter
    outside_set = ~in_set

    # Use log-scaled colors
    colors = np.log(iterations + 1)

    plt.figure(figsize=(20, 15), dpi=150)

    # Plot points inside the set in black
    plt.scatter(complex_points[in_set].real, complex_points[in_set].imag, color='black', s=0.5, marker='.')

    # Plot escaping points with a color gradient
    plt.scatter(complex_points[outside_set].real, complex_points[outside_set].imag, 
                c=colors[outside_set], cmap='inferno', s=0.1, marker='.')

    # Add color bar and other plot details
    plt.colorbar(label="Escape Time (log scale)")
    plt.xlim(x_range)
    plt.ylim(y_range)
    plt.title(f"Mandelbrot Set with {n} Points ({sampler})")
    plt.xlabel("Real")
    plt.ylabel("Imaginary")
    plt.show()


In [ ]:
plot_mandelbrot(1000000, sampler='pure_random_sampling')

# Plot Mandelbrot set using Latin Hypercube Sampling
plot_mandelbrot(n=10000000, x_range=(-2,1), y_range=(-1.5, 1.5), max_iter=1000, sampler='latin_hypercube_sampling')

# Plot Mandelbrot set using Orthogonal Sampling
# Ensure n is a square of a prime number, such as 49 (7^2) or 25 (5^2)
plot_mandelbrot(n=1018081, x_range=(-2,1), y_range=(-1.5, 1.5), max_iter=1000, sampler='orthogonal_sampling')